# 📊 Unión y Limpieza de datos del Dataset  

---

**Objetivo del Notebook**  
Limpieza de datos, columnas innecesarias y valores nulos/blancos 

**Contexto del análisis**  
- Dataset de muestra proporcionado + csv proporcionado unido en un único excel dataset
- Enfoque en aprendizaje, validación del pipeline y comprensión del proceso

**Valor devuelto**  
- Copia del Dataset de muestra proporcionado completamente limpio y útil 

---




In [ ]:
import pandas as pd
import xlsxwriter
import utils
# ===============================
# LEER EL ARCHIVO LIMPIO
# ===============================
dfs = pd.read_excel(r'..\datos\01. Datos originales\DataSET_SF.xlsx', sheet_name=None)

# Ver la primera hoja
oportunidad = list(dfs.values())[0]
cuenta = list(dfs.values())[1]
ecb = list(dfs.values())[2]
solicitud_ban = list(dfs.values())[3]
casos = list(dfs.values())[4]
correos = list(dfs.values())[5]
historial_actividad = list(dfs.values())[6]
historial_etapas = list(dfs.values())[7]


print(historial_etapas.head())

In [ ]:
from utils import analisis_na_por_columna

# ===============================
# ANALISIS POR COLUMNA DE VALORES NA
# ===============================
analisis_na_por_columna(oportunidad)
analisis_na_por_columna(cuenta)
analisis_na_por_columna(ecb)
analisis_na_por_columna(solicitud_ban)
analisis_na_por_columna(casos)
analisis_na_por_columna(correos)
analisis_na_por_columna(historial_actividad)
analisis_na_por_columna(historial_etapas)

In [ ]:
from utils import analisis_na_por_columna, eliminar_columnas_na, crear_target


# ===============================
# LIMPIEZA DE NAS
# ===============================

oportunidad = eliminar_columnas_na(oportunidad)
cuenta = eliminar_columnas_na(cuenta)
ecb = eliminar_columnas_na(ecb)
solicitud_ban = eliminar_columnas_na(solicitud_ban)
casos = eliminar_columnas_na(casos)
correos = eliminar_columnas_na(correos)
historial_actvidad = eliminar_columnas_na(historial_actividad)
historial_etapas = eliminar_columnas_na(historial_etapas)

# ===============================
# CREACION DEL TARJET
# ===============================

target = crear_target(oportunidad, historial_etapas)
target.head()




In [ ]:
# ===============================
# COLUMNAS A TRATAR
# ===============================

columnas_seleccionadas = [
    # Identificadores y Target
    'ACCOUNTID', 'ID','ID18__PC', 'target', 'PL_CURSO_ACADEMICO',
    
    # Rendimiento Académico
    'NU_NOTA_MEDIA_ADMISION', 'NU_NOTA_MEDIA_1_BACH__PC', 'CH_PRUEBAS_CALIFICADAS', 
    'NU_RESULTADO_ADMISION_PUNTOS', 'PL_RESOLUCION_DEFINITIVA', 'TITULACION',
    
    # Compromiso Económico
    'MINIMUMPAYMENTPAYED', 'PAID_AMOUNT', 'PAID_PERCENT', 'CH_PAGO_SUPERIOR', 
    'CH_MATRICULA_SUJETA_BECA', 'CH_AYUDA_FINANCIACION', 'CU_IMPORTE_TOTAL',
    
    # Engagement y Actividad
    'CH_VISITACAMPUS__PC', 'CH_ENTREVISTA_PERSONAL__PC', 'ACC_DTT_FECHAULTIMAACTIVIDAD', 
    'NU_PREFERENCIA', 'STAGENAME', 'PL_SUBETAPA',
    
    # Perfil Sociodemográfico y Fidelidad
    'CH_HIJO_EMPLEADO__PC', 'CH_HIJO_ANTIGUO_ALUMNO__PC', 'CH_HERMANOS_ESTUDIANDO_UNAV__P', 
    'YEARPERSONBIRTHDATE', 'NAMEX', 'CH_FAMILIA_NUMEROSA__PC', 'PL_SITUACION_SOCIO_ECONOMICA',
    
    # Origen y Canal
    'LEADSOURCE', 'PL_ORIGEN_DE_SOLICITUD', 'LK_CENTROENSENANZA', 'PL_PLAZO_ADMISION'
]

# =================================
# Unir la tabla con cuenta y obtener filtrar los datos que nos interesen
# ===============================
df_unido = pd.merge(
    target, 
    cuenta, 
    left_on='ACCOUNTID', 
    right_on='ID18', 
    how='left',
    suffixes=('', '_cuenta')
)		
columnas_finales =  [col for col in columnas_seleccionadas if col in df_unido.columns]
df_unido_filtrado =  df_unido[columnas_finales].copy()


In [ ]:
from utils import calcular_tiempos_etapas, limpiar_historial_por_hitos
import numpy as np
import pandas as pd

historial_etapas_tiempo = calcular_tiempos_etapas(historial_etapas)

# Unir target con historial_etapas y calcular el tiempo medio por etapa
df_unido_filtrado 


df_final = limpiar_historial_por_hitos(historial_etapas_tiempo, df_unido_filtrado)
# Unir la tabla con ECB y pagos


ejemplo_id = '0066900001k7yTgAAI'

columnas_comprobacion = [
    'LK_Oportunidad__c', 'CreatedDate', 'PL_Etapa__c', 'PL_Subetapa__c',
    'fecha_pruebas_calificadas', 'NU_NOTA_MEDIA_ADMISION',
    'fecha_matricula_iniciada', 'PAID_AMOUNT','NU_NOTA_MEDIA_ADMISION', 'CH_PRUEBAS_CALIFICADAS', 
        'NU_RESULTADO_ADMISION_PUNTOS', 'PL_RESOLUCION_DEFINITIVA'
]

print("--- COMPROBACIÓN DE LÓGICA TEMPORAL ---")
df_final[df_final['LK_Oportunidad__c'] == ejemplo_id][columnas_comprobacion].sort_values('CreatedDate')
df_final.head(50)

In [ ]:
import pandas as pd
import numpy as np
from utils import integrar_actividades_progresivo_por_curso

# Ejecución
df_final_v3 = integrar_actividades_progresivo_por_curso(df_final, historial_actividad)
# ==========================================
# EJECUCIÓN
# ==========================================
# Print de comprobación para ver la evolución de un contacto
ejemplo_acc = df_final_v3[df_final_v3['num_asistencias_acum'] > 0]['ID18__PC'].iloc[1]
cols_print = ['ID','ID18__PC','ACCOUNTID', 'CreatedDate', 'PL_Etapa__c', 'num_asistencias_acum', 'num_solicitudes_acum']

print("\n--- COMPROBACIÓN DE EVOLUCIÓN DE ACTIVIDADES ---")
df_final_v3[df_final_v3['ID18__PC'] == ejemplo_acc][cols_print].sort_values('CreatedDate')
historial_actividad.loc[historial_actividad['ContactId']=='003690000312P6pAAE',]

In [ ]:
import pandas as pd

def contar_asistencias_por_fecha(df_actividad, id_col='ContactId'):
    # 1. Asegurar formato datetime
    df_actividad['CreatedDate'] = pd.to_datetime(df_actividad['CreatedDate'], errors='coerce')
    
    # 2. Crear columna de fecha sin hora
    df_actividad['Fecha'] = df_actividad['CreatedDate'].dt.date
    
    # 3. Contar asistencias y no asistencias por alumno y fecha
    resumen = df_actividad.groupby([id_col, 'Fecha']).agg(
        num_asistencias = ('FO_Asiste__c', lambda x: (x == True).sum()),
        num_no_asistencias = ('FO_Asiste__c', lambda x: (x == False).sum())
    ).reset_index()
    
    return resumen
contar_asistencias_por_fecha(historial_actividad).head()

In [ ]:
#FUNCION PARA COMPROBAR LOS DATOS DE LOS CRUCES Y DATAFRAMES
import pandas as pd
import numpy as np
import inspect

def comprobar_cruce_generico(df, clave_principal):
    # Obtener el nombre de la variable que se pasó como argumento
    nombre_df = None
    frame = inspect.currentframe().f_back
    for var_name, var_val in frame.f_locals.items():
        if var_val is df:
            nombre_df = var_name
            break
    if nombre_df is None:
        nombre_df = "DataFrame"
    
    print("\n" + "="*60)
    print(f"🔍 Comprobación de cruce para {nombre_df} ({len(df)} filas, {df.shape[1]} columnas)")
    print("="*60)
    
    # 1. Duplicados y nulos en clave principal
    if clave_principal in df.columns:
        duplicados = int(df[clave_principal].duplicated().sum())
        nulos_clave = int(df[clave_principal].isna().sum())
        print(f"📌 Clave principal: {clave_principal}")
        print(f"   ➡ Duplicados en clave: {duplicados}")
        print(f"   ➡ Nulos en clave: {nulos_clave}")
    else:
        print(f"⚠ La columna clave '{clave_principal}' no existe en el DataFrame")
        return
    
    # 2. Nulos por columna (solo las que tienen nulos)
    nulos_por_col = df.isna().sum()
    nulos_por_col = nulos_por_col[nulos_por_col > 0]
    if not nulos_por_col.empty:
        print("\n📊 Columnas con valores nulos:")
        display(pd.DataFrame(nulos_por_col, columns=['Nulos']))
    else:
        print("\n✅ No hay columnas con valores nulos")
    
    # 3. Incoherencias en columnas numéricas (ej. porcentajes fuera de 0-100)
    incoherencias = {}
    for col in df.select_dtypes(include=[np.number]).columns:
        serie = df[col].dropna()
        if not serie.empty and serie.between(0, 100).sum() != len(serie):
            incoherencias[col] = int(((serie < 0) | (serie > 100)).sum())
    if incoherencias:
        print("\n⚠ Incoherencias en columnas numéricas (fuera de 0-100):")
        display(pd.DataFrame.from_dict(incoherencias, orient='index', columns=['Valores fuera de rango']))
    else:
        print("\n✅ No hay incoherencias numéricas detectadas")
    
    # 4. Fechas incoherentes (si hay al menos 2 columnas datetime)
    columnas_fecha = df.select_dtypes(include=['datetime64[ns]']).columns.tolist()
    if len(columnas_fecha) >= 2:
        fecha_inicio = columnas_fecha[0]
        fecha_fin = columnas_fecha[1]
        incoherentes_fechas = int((df[fecha_fin] < df[fecha_inicio]).sum())
        print(f"\n📅 Comparación de fechas ({fecha_inicio} vs {fecha_fin}):")
        print(f"   ➡ Registros con fecha fin < fecha inicio: {incoherentes_fechas}")
    else:
        print("\nℹ No hay suficientes columnas de fecha para comparar")
    
    print("="*60 + "\n")


# Comprobar cualquier DataFrame (modificar a mano)
resultado_df_unido = comprobar_cruce_generico(df_unido, clave_principal='ID')
resultado_df_account = comprobar_cruce_generico(df_unido_account, clave_principal='ID')
resultado_df_ecb_ban = comprobar_cruce_generico(df_unido_ecb_ban, clave_principal='ID')

print(resultado_df_unido)
print(resultado_df_account)
print(resultado_df_ecb_ban)


In [ ]:
# ===============================
# Tarea 1: Unir historial de etapas y calcular la "Velocidad de Embudo"
# ===============================

# 1. Unir por Id de oportunidad
df_unido = pd.merge(
    target,
    historial_etapas,
    left_on='ID',                 # Id de oportunidad en target
    right_on='LK_Oportunidad__c', # Id de oportunidad en historial_etapas
    how='left',
    suffixes=('', '_historial_etapas')
)

# 2. Eliminar columnas con demasiados NA
df_unido = eliminar_columnas_na(df_unido)

# 3. Convertir fechas a datetime
df_unido['CreatedDate'] = pd.to_datetime(df_unido['CreatedDate'], errors='coerce')
df_unido['Fecha_fin_etapa__c'] = pd.to_datetime(df_unido['Fecha_fin_etapa__c'], errors='coerce')

# 4. Ordenar por oportunidad y fecha de inicio de etapa
df_unido = df_unido.sort_values(by=['LK_Oportunidad__c', 'CreatedDate'])

# 5. Calcular duración de cada etapa
df_unido['DuracionEtapaDias'] = (df_unido['Fecha_fin_etapa__c'] - df_unido['CreatedDate']).dt.days

# 6. Calcular velocidad de embudo (días entre inicio de una etapa y la siguiente)
df_unido['VelocidadEmbudo'] = df_unido.groupby('LK_Oportunidad__c')['CreatedDate'].diff().dt.days

# 7. Resultado
print(df_unido[['LK_Oportunidad__c', 'PL_Etapa__c', 'CreatedDate', 'Fecha_fin_etapa__c', 'DuracionEtapaDias', 'VelocidadEmbudo']].head())




In [ ]:
# ===============================
# Tarea 2: Unir Account y extraer Procedencia, Notas y Relación UNAV
# ===============================

# Columnas necesarias según el listado real
columnas_cuenta = [
    'ID18',  # Id de cuenta para unir
    # Procedencia
    'CH_NACIONAL__PC', 'CH_INTERNACIONAL__PC', 'FO_PAIS__PC', 'FO_PROVINCIA__PC', 'TX_OTRALOCALIDAD__PC',
    # Notas
    'NU_NOTA_MEDIA_1_BACH__PC', 'NU_NOTA_MEDIA_2_BACH__PC', 'NU_NOTA_MEDIA_DEFINITIVA__PC',
    # Relación UNAV
    'CH_ANTIGUO_ALUMNO__PC', 'CH_EMPLEADO__PC', 'CH_PROFESOR_ASOCIADO__PC',
    'CH_HIJO_EMPLEADO__PC', 'CH_HIJO_ANTIGUO_ALUMNO__PC', 'CH_HERMANOS_ESTUDIANDO_UNAV__P', 'CH_ALUMNI__PC'
]

# Filtrar solo las columnas que existen
columnas_existentes = [col for col in columnas_cuenta if col in cuenta.columns]
cuenta_filtrada = cuenta[columnas_existentes]

# Unir con target usando el Id de cuenta
df_unido_account = pd.merge(
    target,
    cuenta_filtrada,
    left_on='ACCOUNTID',  # Id de cuenta en target
    right_on='ID18',      # Id de cuenta en cuenta
    how='left'
)

# Resultado
print(df_unido_account.head())



In [ ]:
# ===============================
# Tarea 3: Unir ECB/BAN y calcular métricas económicas
# ===============================

# 1. Seleccionar columnas de oportunidad
columnas_oportunidad = [
    'ID',  # Id de oportunidad para unir
    'CU_IMPORTE_TOTAL', 'PAID_AMOUNT', 'PAID_PERCENT',
    'LK_DESCUENTO_PRONTO_PAGO', 'NU_IMPORTE_DESCUENTO_PRONTO_PA', 'NU_PORCENTAJE_DESCUENTO_PRONTO'
]
oportunidad_filtrada = oportunidad[[col for col in columnas_oportunidad if col in oportunidad.columns]]

# 2. Seleccionar columnas de solicitud BAN
columnas_ban = [
    'SOL_LK_oportunidad__c', 'SOL_MON_importeConcedido__c'
]
solicitud_ban_filtrada = solicitud_ban[columnas_ban]

# 3. Unir oportunidad con solicitud BAN
df_unido_ecb_ban = pd.merge(
    oportunidad_filtrada,
    solicitud_ban_filtrada,
    left_on='ID',
    right_on='SOL_LK_oportunidad__c',
    how='left'
)

# 4. Calcular métricas
df_unido_ecb_ban['Total_a_pagar'] = df_unido_ecb_ban['CU_IMPORTE_TOTAL']
df_unido_ecb_ban['Porcentaje_pagado'] = df_unido_ecb_ban['PAID_PERCENT']
df_unido_ecb_ban['Beca_concedida'] = df_unido_ecb_ban['SOL_MON_importeConcedido__c']
df_unido_ecb_ban['Importe_descuento'] = df_unido_ecb_ban['NU_IMPORTE_DESCUENTO_PRONTO_PA']
df_unido_ecb_ban['Porcentaje_descuento'] = df_unido_ecb_ban['NU_PORCENTAJE_DESCUENTO_PRONTO']

# 5. Resultado
print(df_unido_ecb_ban[['ID', 'Total_a_pagar', 'PAID_AMOUNT', 'Porcentaje_pagado', 'Beca_concedida', 'Importe_descuento', 'Porcentaje_descuento']].head())



In [ ]:
# ===============================
# Tarea 4: Cruzar Actividades: nº de eventos asistidos por etapa (excluyendo futuros).
# ===============================
from datetime import datetime

# 1. Filtrar actividades excluyendo futuros
actividades_filtradas = historial_actividad.copy()
actividades_filtradas['CreatedDate'] = pd.to_datetime(actividades_filtradas['CreatedDate'], errors='coerce')
actividades_filtradas['CreatedDate'] = actividades_filtradas['CreatedDate'].dt.tz_localize(None)

hoy = pd.Timestamp(datetime.today().date())
actividades_filtradas = actividades_filtradas[actividades_filtradas['CreatedDate'] <= hoy]

# Filtrar solo asistencias reales
actividades_filtradas = actividades_filtradas[actividades_filtradas['FO_Asiste__c'] == True]

# 2. Unir actividades con oportunidades usando CampaignId
df_act_op = pd.merge(
    actividades_filtradas,
    oportunidad[['ID', 'CAMPAIGNID']],
    left_on='CampaignId',
    right_on='CAMPAIGNID',
    how='left'
)

# 3. Unir con historial de etapas para traer la etapa
df_actividades_etapas = pd.merge(
    df_act_op,
    historial_etapas[['LK_Oportunidad__c', 'PL_Etapa__c']],
    left_on='ID',
    right_on='LK_Oportunidad__c',
    how='left'
)

# 4. Contar nº de eventos asistidos por etapa
eventos_por_etapa = df_actividades_etapas.groupby(['LK_Oportunidad__c', 'PL_Etapa__c']).size().reset_index(name='Num_eventos_asistidos')

print(eventos_por_etapa.head())


In [ ]:
# ===============================
# Tarea 5: Extraer motivos de baja de la descripción y vinculación con tipos de beca.
# ===============================
import pandas as pd

# 1. Filtrar columnas necesarias de casos
columnas_casos = [
    'LK_Oportunidad__c', 'Description', 'TX_Motivo_cancelacion_beca__c', 
    'TX_MotivoDenegacion__c', 'LK_Beca__c', 'PL_Resolucion_beca__c', 'NU_Importe_Beca_Concedido__c'
]
casos_filtrados = casos[columnas_casos]

# 2. Función para clasificar motivos de baja
def clasificar_motivo(texto):
    if pd.isna(texto):
        return 'Desconocido'
    texto = str(texto).lower()
    if 'econ' in texto:
        return 'Motivo económico'
    elif 'traslad' in texto:
        return 'Traslado a otra universidad'
    elif 'no interes' in texto or 'desist' in texto:
        return 'Falta de interés'
    elif 'aprob' in texto and 'otra' in texto:
        return 'Admitido en otra universidad'
    else:
        return 'Otro'

# 3. Crear columna de motivo de baja a partir de Description
casos_filtrados['Motivo_baja'] = casos_filtrados['Description'].apply(clasificar_motivo)

# 4. Si quieres, también podemos priorizar el motivo estructurado si existe
casos_filtrados['Motivo_baja_final'] = casos_filtrados['TX_Motivo_cancelacion_beca__c'].fillna(casos_filtrados['Motivo_baja'])

# 5. Resultado final
print(casos_filtrados[['LK_Oportunidad__c', 'Motivo_baja_final', 'LK_Beca__c', 'PL_Resolucion_beca__c', 'NU_Importe_Beca_Concedido__c']].head())


In [ ]:
# ===============================
# Tarea 6: Comprobar que en los cruces no se generan valores que no deben
# ===============================

def comprobar_cruces(df, clave_principal):
    print("=== Comprobación de cruces ===")
    
    # 1. Duplicados en clave principal
    duplicados = df[df[clave_principal].duplicated()]
    if not duplicados.empty:
        print(f"⚠ Hay {len(duplicados)} duplicados en la clave {clave_principal}")
    else:
        print("✅ No hay duplicados en la clave principal")
    
    # 2. Nulos en clave principal
    nulos_clave = df[df[clave_principal].isna()]
    if not nulos_clave.empty:
        print(f"⚠ Hay {len(nulos_clave)} registros con clave principal nula")
    else:
        print("✅ No hay nulos en la clave principal")
    
    # 3. Valores incoherentes en métricas económicas
    if 'Porcentaje_pagado' in df.columns:
        incoherentes_pago = df[(df['Porcentaje_pagado'] < 0) | (df['Porcentaje_pagado'] > 100)]
        if not incoherentes_pago.empty:
            print(f"⚠ Hay {len(incoherentes_pago)} registros con % pagado fuera de rango")
        else:
            print("✅ % pagado dentro de rango")
    
    if 'Beca_concedida' in df.columns and 'Total_a_pagar' in df.columns:
        beca_mayor_total = df[df['Beca_concedida'] > df['Total_a_pagar']]
        if not beca_mayor_total.empty:
            print(f"⚠ Hay {len(beca_mayor_total)} registros con beca mayor que el total a pagar")
        else:
            print("✅ Becas dentro de rango")
    
    # 4. Fechas incoherentes
    if 'CreatedDate' in df.columns and 'Fecha_fin_etapa__c' in df.columns:
        fechas_incoherentes = df[df['Fecha_fin_etapa__c'] < df['CreatedDate']]
        if not fechas_incoherentes.empty:
            print(f"⚠ Hay {len(fechas_incoherentes)} registros con fecha fin anterior a fecha inicio")
        else:
            print("✅ Fechas coherentes")

# Ejemplo de uso con tu DataFrame final
# Comprobar cruces sobre el DataFrame que quieras
# 1. target + historial_etapas
comprobar_cruces(df_unido, clave_principal='ID')

# 2. target + cuenta
comprobar_cruces(df_unido_account, clave_principal='ID')

# 3. oportunidad + solicitud BAN
comprobar_cruces(df_unido_ecb_ban, clave_principal='ID')

# 4. actividades + oportunidad (CampaignId)
comprobar_cruces(df_act_op, clave_principal='ID')

# 5. actividades + oportunidad + historial_etapas
comprobar_cruces(df_actividades_etapas, clave_principal='ID')



Reescribir y crear el dataset nuevo completamente optimizado

In [ ]:
# ===============================
# 3. GUARDAR EXCEL LIMPIO
# ===============================
with pd.ExcelWriter(
    r"..\datos\02. Datos tratamiento preliminar\DataSet Probabilidad Matrículas_Completo.xlsx",
    engine='xlsxwriter'
) as writer:
    for nombre_hoja, df in hojas_limpias.items():
        df_final_v3.to_excel(writer, sheet_name=nombre_hoja, index=False)

print("Archivo limpio guardado como 'DataSet Probabilidad Matrículas_Completo.xlsx'")